#### Exercise 4.1

Implement all the proceeding functions of `Option`.

In [1]:
enum Option[+A]:
    case Some(get: A)
    case None

    def map[B] (f: A => B): Option[B] =
        this match
            case Some(a) => Some(f(a))
            case None => None

    def flatMap[B] (f: A => Option[B]): Option[B] =
        this.map(f).getOrElse(None)

    def getOrElse[B >: A](default: => B): B =
        this match
            case Some(a) => a
            case None => default

    def orElse[B >: A](ob: => Option[B]): Option[B] =
        this.map(a => Some(a)).getOrElse(ob)

    def filter(f: A => Boolean): Option[A] =
        if this.map(f).getOrElse(false) then this else None

defined class Option

In [2]:
Some(1).filter(_ > 1)

res1: Option[Int] = None

#### Exercise 4.2

Implement the `variance` function in terms of `flatMap`. If the means of a sequence is `m`, the variance is the mean of `math.pow(x - m, 2)` for each element in the sequence.

In [3]:
import Option.*

def mean(xs: Seq[Double]): Option[Double] =
    if !xs.isEmpty then Some(xs.sum / xs.length) else None

def variance(xs: Seq[Double]): Option[Double] =
    mean(xs)
        .map(m => xs.map(x => math.pow(x - m, 2)))
        .flatMap(xs => mean(xs))


import Option.*


defined function mean
defined function variance

#### Exercise 4.3

Write a generic function `map2` that combines two `Option` values using a binary function. It either `Option` value is `None`, then the return value is too.

In [4]:
def map2[A, B, C](a: Option[A], b: Option[B])(f: (A, B) => C): Option[C] =
    a.flatMap(a => b.flatMap(b => Some(f(a, b))))

defined function map2

In [5]:
map2(Some(1), Some(2))(_ + _)
map2(Some(1), None : Option[Int])(_ + _)

res4_0: Option[Int] = Some(get = 3)
res4_1: Option[Int] = None

#### Exercise 4.4

Write a function `sequence` that combines a list of `Options` into one `Option` containing a list of all the `Some` values in the original list. If the original list contains `None` even once, the result of the function should be `None`; otherwise, the result should be `Some`, with a list of all the values.

In [12]:
def sequence[A](as: List[Option[A]]): Option[List[A]] =
    as.foldLeft (Some(List()): Option[List[A]]) (map2(_, _)((bs, a) => a :: bs))


defined function sequence

In [14]:
sequence(List(Some(1), Some(2), Some(3)))
sequence(List(Some(1), None, Some(3)))

res13_0: Option[List[Int]] = Some(get = List(3, 2, 1))
res13_1: Option[List[Int]] = None

#### Exercise 4.5

Implement the function `traverse`. It's straightforward to do using `map` and `sequence`, but try for a more efficent implementation that only looks at the list once. In fact, implement `sequence` in terms of `traverse`.

In [17]:
def traverse[A, B](as: List[A])(f: A => Option[B]): Option[List[B]] =
    as.foldRight[Option[List[B]]](Some(Nil))((a, bs) => map2(f(a), bs)(_ :: _))

def sequence[A](as: List[Option[A]]): Option[List[A]] =
    traverse(as)(identity)

defined function traverse
defined function sequence

In [19]:
def toIntOption(s: String): Option[Int] =
    try Some(s.toInt)
    catch case _ : NumberFormatException => None

traverse(List("1", "2"))(toIntOption)
traverse(List("1", "a"))(toIntOption)

defined function toIntOption
res18_1: Option[List[Int]] = Some(get = List(1, 2))
res18_2: Option[List[Int]] = None

In [20]:
sequence(List(Some(1), Some(2), Some(3)))
sequence(List(Some(1), None, Some(3)))

res19_0: Option[List[Int]] = Some(get = List(1, 2, 3))
res19_1: Option[List[Int]] = None

#### Exercise 4.6

Implement versions of `map`, `flatMap`, `orElse`, and `map2` on `Either` that operate on the `Right` value.

In [3]:
enum Either[+E, +A]:
    case Left(value: E)
    case Right(value: A)

    def map[B](f: A => B): Either[E, B] = this match
        case Right(a) => Right(f(a))
        case Left(e) => Left(e)
    
    def flatMap[EE >: E, B](f: A => Either[EE, B]): Either[EE, B] = this match
        case Right(a) => f(a)
        case Left(e) => Left(e)

    def orElse[EE >: E, B >: A](b: => Either[EE, B]): Either[EE, B] = this match
        case Left(_) => b
        case _ => this
    
    def map2[EE >: E, B, C](that: Either[EE, B])(f: (A, B) => C): Either[EE, C] =
        this.flatMap(a => that.map(b => f(a, b)))


import scala.util.control.NonFatal

object Either:
    def catchNonFatal[A](a: => A): Either[Throwable, A] =
        try Right(a)
        catch case NonFatal(t) => Left(t)

defined class Either
import scala.util.control.NonFatal


defined object Either

#### Exercise 4.7

Implement `sequence` and `traverse` for `Either`. These should return the first error that's encountered if there is one:

In [5]:
import Either.*

def traverse[E, A, B](as: List[A])(f: A => Either[E, B]): Either[E, List[B]] =
    as.foldRight[Either[E, List[B]]](Right(Nil))(f(_).map2(_)(_ :: _))

def sequence[E, A](as: List[Either[E, A]]): Either[E, List[A]] =
    traverse(as)(identity)


import Either.*


defined function traverse
defined function sequence

#### Note

It is interesting to see how the implementation of `map`, `flatMap`, `orElse` and `map2` are so similar between `Option` and `Either`. And, to see how the impementations of `traverse` are nearly identical and that `sequence` is in fact the same.

It is also interesting to note how closely related `lift` and `map2` are that there are hints that additional machinery is required to thread the two more generally.

Given, how closely related the implementation of `map2` and `traverse` are, I wonder what similar concepts for `List` would mean or even look like.